In [ ]:
import matplotlib.pyplot as plt
from utils_detectron import DetectronModel

# Initialize the model
model = DetectronModel()

# Build training configuration
model.build_train_cfg()

# Train the model
model.train()

# Evaluate the trained model
model.build_eval_cfg()
results = model.evaluate()
print(results)

# Inference on a test image
model.build_predict_cfg()
image_path = "scarecrow_coco_dataset/test/test_original_13.png"
output_image = model.predict_and_visualize(image_path)

# Display the output image
plt.figure(figsize=(12, 8))
plt.imshow(output_image)
plt.axis("off")
plt.show()
